# Main logical components

### data grabber
- real cameras rarely produce data in some convenient representation, like popular image formats, and there become in use low level approaches like TCP/UDP, shared memory to transfer data. It is reasonable, cause cameras could produce enormous amount of data and work on big frame rates, like 100 or even 500 Hz or produce high accuracy data, like 32 bits.

So, there should be component, that could grab data and represent it as something convenient in Python, example np.array.

def grab() -> point_cloud (np.array), metadata (dict)

### inference
- component, that performs analysis: object detection, segmentation - not matters. It heavily use computation. It sould be done in the way, to enable all CPU-s and GPU-s, it case of Python it means usage of Multiprocessing, and queues and shared memory as much as possible. If we have cloud environment, there could be lot's of inference servers, and something to balance loading, lake RabbitMQ, python code, that split large task to smaller ones and than merge result.

def infer(point_cloud, metadata) -> result (np.array), metadata (dict)

### engine
- orchestrates work of data grabber and inference, to make possible to fastly build similar APIs on different protocols. Like we could have HTTP, GRPS, just low level TCP/UDP API-s. It is like internal low-level API.

def run_processing(request) -> response(result, metadata)

### API
- it could be HTTP (Flask, FastAPI, etc), GRPC or anything else, not much matters. It validates user request, check engine status, pass request to engine, validate results, send results back to user.

def grab_latest_frame_and_detect_objects() -> result, metadata

# Problems that may arise during the deployment

- deployment on different operating systems and CPU arcitectures could couse recompilation of some components and eaven there are situations, when components, only work with particular x64 or ARM arcitecture (same with operating systems). So, make as less dependencies and keep it in mind.
- computer vision is very computation intensive and we should keep in mind, that on machine run not only object detection. So, communicate with other team members and estimate how much resources could consume every component, to not run-out RAM or CPU.
- inter-process, inter-component communication. In the new environment could be not correctly configured network or file access, and we should make easy to use tools to check how data flows.
- users in much cases need point cloud and analytics to be synchronized and we should carefully configure system to not overflow buffers and queues, and to avoid interlocks.
- consider security issues, that deployment could be done in offline mode, and our Python solutions should be done in the way, to not be easy grabbed, to prevent stealing our algorithms.